In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

In [3]:
url='https://www.cdslab.org/recipes/programming/stat-covmat/globalLandTempHist.txt'

In [4]:
#downloads file from given url
def download(url, fpath):
    import requests
    r = requests.get(url)
    with open(fpath, 'wb') as f:
        f.write(r.content)
    return None

#fetching data file
def fetchDatafromWeb(filePath="./globalLandTempHist.txt",
                         url= url):
    download(url,filePath)
    return filePath

def readFile(fpath):
    import pandas as pd
    return pd.read_csv(fpath, ", ") 


In [6]:
fetchDatafromWeb()

'./globalLandTempHist.txt'

In [5]:
df=readFile('./globalLandTempHist.txt')
df

<ipython-input-4-47e34b36e366>:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(fpath, ", ")


year  anomaly
0     1743.846154   -1.131
1     1743.923077      NaN
2     1744.076923      NaN
3     1744.153846      NaN
4     1744.230769      NaN
...           ...      ...
3234  2013.384615    0.813
3235  2013.461538    1.088
3236  2013.538462    0.671
3237  2013.615385    0.919
3238  2013.692308    1.950

[3239 rows x 2 columns]

In [6]:
#slicing dataframe to get rows 109-3238
sdf=df.loc[109:]

In [7]:
#droping rows with values NaN
ndf=df.dropna()
ndf

year  anomaly
0     1743.846154   -1.131
5     1744.307692    1.926
6     1744.384615   -1.115
7     1744.461538   -0.235
8     1744.538462   -0.430
...           ...      ...
3234  2013.384615    0.813
3235  2013.461538    1.088
3236  2013.538462    0.671
3237  2013.615385    0.919
3238  2013.692308    1.950

[3167 rows x 2 columns]

# Computing covariance and correlation matrix

In [123]:
# calculates the covariance between X and y from given data
def cov(x, y):
    mean_x, mean_y = x.mean(), y.mean()
    n = len(x)
    
    return sum((x - mean_x) * (y - mean_y)) / (n-1)
 
    
# calculates the Covariance matrix for input data 
def genCovMat(data, Mean=None):
    
    """""
    Generate and return the covariance matrix of input data.
    The cols of the data must be individual values.
    The rows must be individual observations.
    Please pass clean matrix of all real values (0,NA, no NaN).
    
    Parameters
    ------------
        Data
            The input numpy matrix of data of all numeric values.
        Mean
            The mean of the input data along the columns (Attributes)
            (**optional**, default=np.mean(data))
        
    """""
 # get the rows and cols
    rows, cols = data.shape
    
 # the covariance matrix has a shape of n_attributes x n_attributes
    cov_mat = np.zeros((cols, cols))
    #nfact= 1/(len(data[:,0])-1)
    for i in range(cols):
        #for j in range (i+1):
        for j in range(cols):
                # store the value in the matrix
                cov_mat[i][j] = cov(data[:, i], data[:, j]) 
                #cov_mat[i][j] = nfact*np.dot(data[:,i]-mean[i],data[:,j]-mean[j])
    return cov_mat

# calculates Correlation matrix using Covariance function (cov()) for input data
def genCorMat(data):

    rows, cols = data.shape
    #corr_mat = np.ones((cols, cols))
    corr_mat = np.ones((cols, cols))
    
    for i in range(cols):
        #for j in range (i+1): 
        for j in range(cols): 
            #if i/=j: not equal

                x, y = data[:, i], data[:, j]
                # note: covariance matrix is just being normalized
                #corr_mat[i][j] = cov(x, y) / (x.std() * y.std())
                corr_mat[i,j] = data[i,j] / np.sqrt(data[i,i]*data[j,j])
                #corr_mat[j,i] = corr_mat[i,j] 

    return corr_mat

In [102]:
data=ndf.to_numpy()
data_sliced=sdf.to_numpy()

In [124]:
#using dropna
cov_matrix=genCovMat(data)
corr_matrix=genCorMat(cov_matrix)
print(cov_matrix)
print(corr_matrix)

[[5.81382831e+03 2.30956353e+01]
 [2.30956353e+01 8.75891017e-01]]
[[1.         0.32364863]
 [0.32364863 1.        ]]


In [147]:
#using df slice after row 109
cov_smatrix=genCovMat(data_sliced)
corr_smatrix=genCorMat(cov_smatrix)
print(cov_smatrix)
print(corr_smatrix)

[[5.67131156e+03 2.30733711e+01]
 [2.30733711e+01 8.70280941e-01]]
[[1.         0.32842754]
 [0.32842754 1.        ]]


# Computing Pearson correlation coefficient

In [72]:
import math
def getPearsonCor(data1,data2):
    """""
        Generate and return the pearson correlation coefficient. 
        Input two columns of data to your function to compute the Pearson corrcoeff. 
        The columns correspond to the data attributes.
        Make sure rows of data with "nan" are removed.
        
        Parameters:
            data1,data2 = data attribute 1, data attribute 2
    """""
    x_mean, y_mean = data1.mean(), data2.mean()
    #x_std, y_std = data1.std(), data2.std()
    x_var = sum ((x_i - x_mean)**2 for x_i in data1)
    y_var = sum ((y_i - y_mean)**2 for y_i in data2)
    
    #cov(x,y)- numerator
    #sqrt(varx*vary)-denominator
    cov = sum((x_i-x_mean)*(y_i-y_mean) for x_i,y_i in zip(data1,data2))
    den = math.sqrt(x_var*y_var)
    
        
    return cov/den

In [73]:
getPearsonCor(data[:,0],data[:,1])

0.3236486317130351

In [81]:
#using numpy
np.corrcoef(data[:,0],data[:,1])

array([[1.        , 0.32364863],
       [0.32364863, 1.        ]])

In [19]:
#using pandas
ndf.corr()

year   anomaly
year     1.000000  0.323649
anomaly  0.323649  1.000000

In [32]:
#using SciPy
#scipy.stats.pearsonr(x,y) returns (corrcoef, p-value)
import scipy.stats
scipy.stats.pearsonr(data[:,0],data[:,1])[0]

0.3236486317130345

Pearson's r value (corrcoef between years and anomaly) is greater than 0.
This means it has positive correlation that indicates that the global land temperature has increased with time over the past 300 years.

# Computing the Spearman's correlation coefficinet of a dataset

In [144]:
def getRank(vector):
    lst= sorted(range(len(vector)), key=vector.__getitem__)
    return np.asarray(lst, dtype=np.float32)

In [145]:
def getSpearmanCor(data1,data2):
    """""
        Generate and return the spearman correlation coefficient. 
        Input two columns of data to your function to compute the spearman corrcoeff. 
        The columns correspond to the data attributes.
        Make sure rows of data with "nan" are removed.
        
        Parameters:
            data1,data2 = data attribute 1, data attribute 2
    """""
    
    #get array ranks of X,y
    X_rank = getRank(data1)
    Y_rank = getRank(data2)
    
    xr_mean, yr_mean =  X_rank.mean(),  Y_rank.mean()
   

    xr_var = sum ((x_i - xr_mean)**2 for x_i in  X_rank)
    yr_var = sum ((y_i - yr_mean)**2 for y_i in  Y_rank)
    
    #cov(x,y)- numerator
    #sqrt(varx*vary)-denominator
    cov = sum((x_i-xr_mean)*(y_i-yr_mean) for x_i,y_i in zip( X_rank, Y_rank))
    den = math.sqrt(xr_var*yr_var)
    #den = x_std*y_std --doesnt work
        
    return cov/den

In [146]:
getSpearmanCor(data[:,0],data[:,1])

0.3848867684561194

In [110]:
 ##scipy.stats.spearmanr(x, y) returns (correlation rho, p-value)
scipy.stats.spearmanr(data[:,0],data[:,1]).correlation

0.38458820147753553

In [111]:
#using pandas
ndf['year'].corr(ndf['anomaly'], method='spearman')

0.38458820147753553

# Computing the Kendall's rank correlation coefficinet of a dataset

A pair of observations (xᵢ, yᵢ) and (xⱼ, yⱼ), where i < j, will be one of three things:
1. concordant if either (xᵢ > xⱼ and yᵢ > yⱼ) or (xᵢ < xⱼ and yᵢ < yⱼ)


2. discordant if either (xᵢ < xⱼ and yᵢ > yⱼ) or (xᵢ > xⱼ and yᵢ < yⱼ)


3. neither if there’s a tie in x (xᵢ = xⱼ) or a tie in y (yᵢ = yⱼ)

In [202]:
def getKendallCor(data1,data2):
    """""
        Generate and return the Kendall's rank correlation coefficient. 
        Input two columns of data to your function to compute the Kendall's rank corrcoeff. 
        The columns correspond to the data attributes.
        Make sure rows of data with "nan" are removed.
        
        Parameters:
            data1,data2 = data attribute 1, data attribute 2
    """""
    C = 0
    D = 0
    extra_y = 0
    extra_x = 0

    for i in range(0, len(data2)):
        for j in range(i+1, len(data1)):
            Y1 = data1[i]
            X1 = data2[i]
            Y2 = data1[j]
            X2 = data2[j]

            if Y1 > Y2 and X1 > X2:
                C += 1
            elif Y1 < Y2 and X1 < X2:
                C += 1
            elif Y1 > Y2 and X1 < X2:
                D += 1
            elif Y1 < Y2 and X1 > X2:
                D += 1
            elif Y1 == Y2:
                extra_y += 1
            elif X1 == X2:
                extra_x += 1
    #n=len(Y)
    #N= n*(n-1)*0.5
    v1= np.sqrt(C+D+extra_y)
    v2= np.sqrt(C+D+extra_x)
    k= (C - D) / (v1*v2)

    return k

In [203]:
getKendallCor(data[:,0],data[:,1])

0.31504920018970456

In [37]:
#scipy.stats.kendalltau(x, y) returns (correlation, p-value)
scipy.stats.kendalltau(data[:,0],data[:,1]).correlation

0.3150492001897045

In [34]:
#using pandas
ndf['year'].corr(ndf['anomaly'], method='kendall')

0.3150492001897045